In [1]:
import os
import gym
import theano
import theano.tensor as T
import numpy as np
import heapq
import ppaquette_gym_doom

In [2]:
# Create a classic Doom environment with Gym
env = gym.make('ppaquette/DoomDefendCenter-v0')
env.reset()

INFO:gym.envs.registration:Making new env: ppaquette/DoomDefendCenter-v0
[2017-03-20 23:10:47,897] Making new env: ppaquette/DoomDefendCenter-v0


array([[[ 0,  0, 35],
        [ 0,  0, 35],
        [ 0,  0, 35],
        ..., 
        [ 0,  0,  0],
        [ 0,  0, 35],
        [ 0,  0, 35]],

       [[ 0,  0, 35],
        [ 0,  0,  0],
        [ 0,  0,  0],
        ..., 
        [ 0,  0, 35],
        [ 0,  0,  0],
        [ 0,  0,  0]],

       [[ 0,  0, 35],
        [ 0,  0,  0],
        [ 0,  0,  0],
        ..., 
        [ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0]],

       ..., 
       [[19, 19, 19],
        [19, 19, 19],
        [27, 27, 27],
        ..., 
        [47, 47, 47],
        [35, 35, 35],
        [35, 35, 35]],

       [[11, 11, 11],
        [11, 11, 11],
        [19, 19, 19],
        ..., 
        [19, 19, 19],
        [19, 19, 19],
        [19, 19, 19]],

       [[11, 11, 11],
        [11, 11, 11],
        [19, 19, 19],
        ..., 
        [19, 19, 19],
        [19, 19, 19],
        [19, 19, 19]]], dtype=uint8)

In [4]:
# Agent
class QAgent:
  def __init__(self, learn_rate=0.05, \
               action_set=set(),
               observation_encoder=lambda x: x,\
               verbose=False):
    self.alpha = 0.9 # Probability of re-learning 
    self.policy = dict() # Q: (state,action) => value
    self.total_reward = 0
    self.learn_rate = learn_rate
    self.action_set = action_set
    self.observation_encoder = observation_encoder
    self.prev_state = 0
    self.verbose = verbose
    
  def learn_aggregate(self, observation, action, reward):
    # Encode [observation] => [state]
    state = self.observation_encoder(observation)
    learn_Q(self.prev_state, state, action, reward)
    self.prev_state = state
    

  """
  Predict the reward we would get
  if take [action] on [state]
  """
  def learn_Q(self, state, state_, action, reward):
    curr_Q  = Q(state, action)
    _,max_Q = find_best_action(state_)
    new_Q   = curr_Q + self.alpha*(reward + self.learn_rate * max_Q - curr_Q)
    if state in self.policy:
      self.policy[state][action] = reward # Always overwrite
    else: self.policy[state] = {action: new_Q}

  """
  Find maximum possible reward we would get
  from the best action attempted on [state]
  """
  def find_best_action(self, state):
    if state in self.policy:
      max_Q    = -1
      best_act = -1
      for a,r in self.policy[state].items():
        if r > maxQ:
          best_act = a
          max_Q = r
      return (best_act, max_Q)
    else: return (-1,-1)

  """
  Get recorded Q value of (state, action)
  """
  def Q(self, state, action):
    if state in self.policy:
      if action in self.policy[state]:
        return self.policy[state][action]
      else: return -1
    else: return -1
  
  @staticmethod
  def load(path, default):
    if os.path.isfile(path):
      with open(path,'rb') as f:
        return pickle.load(f) 
    else:
      print('MODEL NOT FOUND, initialising a brand new one.')
      return default

  @staticmethod
  def save(path,agent):
    with open(path,'wb+') as f:
      return pickle.save(agent, f)



In [1]:
# Play
num_episodes = 1

bot = QAgent.load('../bot/QAgent.pkl', QAgent())

for i in range(num_episodes):
  print('Ep#{} started...'.format(i))
  observation = env.render()
  # random_action = env.action_space.sample()
  observation, reward, done, info = env.step()
  bot.learn_aggregate(observation, reward)
  if done:
    print('[Done]')
    print('...Total reward : {}'.format(bot.total_reward))
    # Save the agent
    Q
    break
    
QAgent.save('../', bot)

NameError: name 'QAgent' is not defined